In [8]:
df1 = pd.read_csv('df1.csv', encoding='euc-kr')
df2 = pd.read_csv('df2.csv', encoding='euc-kr' )
dfX2= pd.read_csv('dfX2.csv', encoding='euc-kr' )
df1_scale=df1.copy()

In [6]:
# OLS회귀분석을 통해 예측할 y값: SalesPerStore
# KNN클러스터링을 하고싶은 더미변수(OLS시 X에 들어갈 변수): Code, ServiceCode
df2.tail(3)

,SalesPerStore,Code,ServiceCode
49412,1.284095e+08,1001010,CS300002
49413,1.666019e+07,1001010,CS300005
49414,2.319462e+06,1001010,CS300016


In [4]:
# OLS회귀분석때 사용할 나머지 X값들
# Robust Scaling을 하고난 값
dfX2.tail(3)


,log_DayWeekend,log_FemaleMaleend,Twenty_30,Six_11,Eleven14,log_Fourteen17,Seventeen21,Twentyone24,AptNum,AptArea,AptPrice,MonthlyIncome,Consume,ServiceSale,StoreNum
49412,-0.015023,-0.745555,0.196687,1.186380,-0.589404,-0.560791,-0.492501,0.631402,0.917722,-0.6,-0.092682,-0.018514,1.082662,1.603892,-0.2
49413,0.543757,0.720642,-1.572804,0.119229,-0.280983,0.528788,0.421652,-0.204323,0.917722,-0.6,-0.092682,-0.018514,1.082662,0.092910,0.4
49414,-0.443124,-0.381838,-0.961705,0.947063,1.455827,-0.110810,-0.650427,-0.466276,0.917722,-0.6,-0.092682,-0.018514,1.082662,-0.399699,-0.2


In [25]:
# 로그 취하기
df1_scale['log_DayWeekend']=np.log(df1['DayWeekend'])
df1_scale['log_FemaleMaleend']=np.log(df1['FemaleMale'])
df1_scale['log_Fourteen17']=np.log(df1['Fourteen17'])
df1_scale=df1_scale[['log_DayWeekend', 'log_FemaleMaleend', 'Twenty_30', 'Six_11',
       'Eleven14',  'log_Fourteen17', 'Seventeen21', 'Twentyone24', 'AptNum',
       'AptArea', 'AptPrice', 'MonthlyIncome', 'Consume', 'ServiceSale',
       'StoreNum']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [46]:
# 다음과 같이 회귀분석햇으나 상권코드(Code)의 갯수가 너무 많아서 다중공선성 문제 발생
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

dfX=df1_scale
rb = RobustScaler()
# rb = StandardScaler()
rb.fit(dfX)
X_robust_scaled = rb.transform(dfX)
dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

# model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)
# result = model.fit()
# # print(result.summary())

In [98]:
#우선 처음에는 상권 코드만 클러스터링 하고자함
# KNN 클러스터링을 하고자 하는데, 데이터가 너무 많아 우선 50개만 가지고 코딩을 해보고자 함
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)

dfX=df1_scale
rb = RobustScaler()
# rb = StandardScaler()
rb.fit(dfX)
X_robust_scaled = rb.transform(dfX)
dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

X= df1_robust_scaled[df1_robust_scaled.columns[4:]].loc[:100]
y= df1_robust_scaled['Code'].loc[:100]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.7, random_state=42)
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=10)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, y_train)

y_pred = nca_pipe.predict(X_test)

# model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)

# result = model.fit()
# print(result.summary())

In [99]:
X_test.tail()

,log_FemaleMaleend,Twenty_30,Six_11,Eleven14,log_Fourteen17,Seventeen21,Twentyone24,AptNum,AptArea,AptPrice,MonthlyIncome,Consume,ServiceSale,StoreNum
56,-0.503334,0.075507,-0.507303,0.426874,-0.317158,0.103634,0.573069,-0.778481,3.5,2.075276,0.591749,-1.488331,4.820679,0.4
50,-0.017835,0.111117,-0.508250,0.383878,-0.110712,-0.071053,0.719189,-0.525316,3.8,1.103848,1.095741,-1.397459,3.145282,1.0
87,0.890040,1.126483,-0.507337,-0.397001,0.629269,0.907215,-0.323087,-0.968354,8.7,5.026186,0.936025,-1.739383,0.525723,-0.4
31,-0.698264,-0.810923,-0.157639,-0.103453,0.646509,-0.134481,0.125172,-1.025316,-0.1,-0.132313,-0.394015,-1.436692,-0.360718,0.8
64,0.291444,0.693618,-0.470905,0.967335,0.426359,-0.100007,-0.316731,-0.962025,6.5,4.055414,0.606685,-1.682912,0.885987,3.4


In [100]:
y_pred

array([1000003, 1000001, 1000007, 1000001, 1000007, 1000001, 1000004,
       1000001, 1000003, 1000007, 1000007, 1000001, 1000001, 1000001,
       1000007, 1000003, 1000007, 1000010, 1000001, 1000003, 1000010,
       1000003, 1000001, 1000003, 1000003, 1000010, 1000003, 1000001,
       1000001, 1000003, 1000003, 1000007, 1000001, 1000010, 1000010,
       1000001, 1000004, 1000007, 1000007, 1000003, 1000004, 1000001,
       1000001, 1000007, 1000007, 1000010, 1000001, 1000007, 1000007,
       1000007, 1000003, 1000001, 1000003, 1000001, 1000007, 1000007,
       1000003, 1000001, 1000001, 1000003, 1000001, 1000007, 1000010,
       1000010, 1000010, 1000001, 1000001, 1000001, 1000007, 1000003,
       1000007], dtype=int64)

In [101]:
np.unique(y_pred)

array([1000001, 1000003, 1000004, 1000007, 1000010], dtype=int64)

In [102]:
np.unique(np.array(y_test))

array([1000001, 1000002, 1000003, 1000004, 1000005, 1000006, 1000007,
       1000008, 1000009, 1000010], dtype=int64)

#### 100개 KNN 실시한 결론
- n_neighbors값을 크게할수록 y_pred의 종류가 작아진다
- 즉 군집화가 되는것 같다.

### n_neighbors를 크게하고 모든 데이터를 넣어서 KNN해보기

In [ ]:
#우선 처음에는 상권 코드만 클러스터링 하고자함
# KNN 클러스터링을 하고자 하는데, 데이터가 너무 많아 부분적으로 시도해보고자함
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)

dfX=df1_scale
rb = RobustScaler()
# rb = StandardScaler()
rb.fit(dfX)
X_robust_scaled = rb.transform(dfX)
dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

start_num = 20000
end_num = 50000
X= df1_robust_scaled[df1_robust_scaled.columns[4:]].loc[start_num:end_num]
y= df1_robust_scaled['Code'].loc[start_num:end_num]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=500)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, y_train)

y_pred = nca_pipe.predict(X_test)

# model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)

# result = model.fit()
# print(result.summary())

In [145]:
len(y)

29415

In [139]:
np.unique(y_pred)

array([1000032, 1000051, 1000064, 1000073, 1000077, 1000079, 1000104,
       1000107, 1000118, 1000119, 1000123, 1000124, 1000128, 1000134,
       1000137, 1000154, 1000159, 1000161, 1000163, 1000165, 1000166,
       1000184, 1000200, 1000207, 1000220, 1000229, 1000866, 1000871,
       1000917, 1000930, 1000941, 1000950, 1000951, 1000954, 1000966,
       1000968, 1000981, 1000993, 1000998], dtype=int64)

In [140]:
np.unique(np.array(y_test))

array([1000001, 1000002, 1000003, 1000004, 1000005, 1000006, 1000007,
       1000008, 1000009, 1000010, 1000011, 1000012, 1000013, 1000014,
       1000015, 1000016, 1000017, 1000018, 1000019, 1000020, 1000021,
       1000022, 1000023, 1000024, 1000025, 1000026, 1000027, 1000028,
       1000029, 1000030, 1000031, 1000032, 1000033, 1000034, 1000035,
       1000036, 1000037, 1000038, 1000040, 1000041, 1000042, 1000043,
       1000044, 1000045, 1000046, 1000047, 1000048, 1000049, 1000050,
       1000051, 1000052, 1000053, 1000054, 1000055, 1000056, 1000057,
       1000058, 1000059, 1000060, 1000061, 1000062, 1000063, 1000064,
       1000065, 1000066, 1000067, 1000068, 1000069, 1000070, 1000071,
       1000072, 1000073, 1000074, 1000075, 1000076, 1000077, 1000078,
       1000079, 1000080, 1000081, 1000082, 1000083, 1000084, 1000085,
       1000086, 1000087, 1000088, 1000089, 1000090, 1000091, 1000092,
       1000093, 1000094, 1000095, 1000096, 1000097, 1000098, 1000099,
       1000100, 1000

### 소결론
- 에러가 뜬다

```
input contains NaN, infinity or a value too large for dtype('float64').

```
또는
```
The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
```
- 부분적으로 실행하면 에러가 나지 않는다

```
X= df1_robust_scaled[df1_robust_scaled.columns[4:]].loc[StartNum:EndNum]
y= df1_robust_scaled['Code'].loc[StartNum:EndNum]
```
상기의 식에서 (StartNum, EndNum)을 
- (100:500) =>성공
- (500:1000) => 성공
- (100:5000) =>성공 (n:100)
- (5000: 10000) =>성공
- (10000: 15000) =>성공
- (20000: 50000) =>실패(n:100, 500) 


- 모든 데이터를 KNN하는 법을 찾아야 한다



## 참고 모델 저장하는법(모델링에 실패해서 지금 저장하지는 않음)
#1. 변수에 저장하고 확인하기
```
import pickle
from sklearn.externals import joblib
X,y =iris.data, iris.target
clf = LogisticRegression(random_state=0)
clf.fit(X,y)

saved_model = pickle.dumps(clf)

```
#모델 불러와서 예측하기
```
#load
clf_from_pickle = pickle.loads(saved_model)

#use model
clf_from_pickle.predict(X)

```

#모델을 파일로 저장하기
```
joblib.dump(clf,'filename.pkl')

```

#저장한 파일 불러와서 예측하기
```
clf_from_joblib = joblib.load('filename.pkl')
clf_from_joblib.predict(X)
```




### 평소대로 OLS 회귀분석하듯이 하고, 중간에 KNN넣어보기
- 에러가 발생했다
```
The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
```
- 아무래도 이 방법은 아닌듯 하다
- 매출액이 군집화 되는 방법은 우리가 원하는 방법이 아닌것 같기 때문

In [103]:

# # from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)

# dfX=df1_scale
# rb = RobustScaler()
# # rb = StandardScaler()
# rb.fit(dfX)
# X_robust_scaled = rb.transform(dfX)
# dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
# df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

# X= df1_robust_scaled[['Code', 'log_DayWeekend', 'log_FemaleMaleend',
#        'Twenty_30', 'Six_11', 'Eleven14', 'log_Fourteen17', 'Seventeen21',
#        'Twentyone24', 'AptNum', 'AptArea', 'AptPrice', 'MonthlyIncome',
#        'Consume', 'ServiceSale', 'StoreNum']].loc[:100]
# y= df1_robust_scaled[['SalesPerStore']].loc[:100]

# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.7, random_state=42)
# nca = NeighborhoodComponentsAnalysis(random_state=42)
# knn = KNeighborsClassifier(n_neighbors=3)
# nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
# nca_pipe.fit(X_train, y_train)

# y_pred = nca_pipe.predict(X_test)

# # model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)

# # result = model.fit()
# # print(result.summary())

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [104]:
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import StandardScaler

# df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

# model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)
# result = model.fit()
# print(result.summary())